In [ ]:
#import modules
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
import sqlalchemy as db
import cx_Oracle as orcl
import re

import numpy as np

print("cx_Oracle.version:", orcl.version)
# show all columns without without truncating
pd.set_option('display.max_columns', None)



In [ ]:
pd.re

host = "10.21.18.15"
port = "1521"
sid = "PROFITSDB"
dsn = orcl.makedsn(host, port, sid)
connection = orcl.connect("mobileuser","Kenya1234##", dsn)
cur = orcl.Cursor(connection) 


loan = '''select * from prod.LOAN_ACCOUNT'''
loan = pd.read_sql_query(loan, connection)
loan

##checking customer information
statements = """SELECT * FROM CUSTOMER"""
customer = pd.read_sql_query(statements, connection)
pd.set_option('display.max_columns', None)
customer.head(4)

###latest customer numbers
customer.tail(4)

loan_accounts = '''select a.CUST_ID, e.FIRST_NAME, e.MIDDLE_NAME,e.SURNAME,e.TELEPHONE_1,a.ACC_SN,a.ACC_CD, a.SHORT_NAME,a.OLD_ACCOUNT_NUM,a.ACC_DESCRIPTION, a.LOAN_STATUS,(CASE WHEN a.ACC_STATUS = '1' THEN 'NORMAL' WHEN a.ACC_STATUS = '2' THEN 'OVERDUE' WHEN a.ACC_STATUS = '3' THEN 'DEFINATE DELAY' WHEN a.ACC_STATUS = '4' THEN 'WRITE OFF' END) as LOAN_STATUS_decode,a.FK_LOANFK_PRODUCTI,a.FKGD_HAS_AS_LOAN_P,a.FK_UNITCODE,a.FKINT_AS_FLOATING,a.FKGD_CATEGORY,a.DEP_ACC_UNIT, a.MAIN_BENEF_CATEGOR,a.UNIT,a.FK_AGREEMENTAGR_SN,a.DEP_ACC_SN,a.TOT_COMMISSION_AMN,a.TOT_DRAWDOWN_AMN,a.ACC_LIMIT_AMN, a.DRAWDOWN_FST_AMN,a.LST_TRX_DT,a.LST_INT_CALC_DT,a.DRAWDOWN_EXP_DT,a.ACC_OPEN_DT,a.ACC_EXP_DT,a.DRAWDOWN_FST_DT,a.TMSTAMP,e.DATE_OF_BIRTH,e.CUST_OPEN_DATE,a.LOAN_STATUS,e.CUST_TYPE
from prod.LOAN_ACCOUNT a, prod.customer e
where a.cust_id = e.cust_id'''
loan_accounts = pd.read_sql_query(loan_accounts, connection)
loan_accounts 

loan_accounts = loan_accounts.query('CUST_TYPE == "1" ')

loans_table = loan_accounts.groupby(['CUST_ID','LOAN_STATUS_DECODE']).agg({'CUST_ID': lambda x: len(x), 'CUST_ID': lambda x: len(x),'FK_LOANFK_PRODUCTI': lambda x: len(x)})
loans_table.rename(columns={'CUST_ID': 'count_of_loans_taken_in_bank',
                           'FK_LOANFK_PRODUCTI': 'count_loan_products'}, inplace=True)

In [83]:
loans_table.reset_index(inplace=True)
loans_table

,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank
0,298,NORMAL,1,1
1,303,NORMAL,2,2
2,305,DEFINATE DELAY,1,1
3,314,NORMAL,1,1
4,505,DEFINATE DELAY,2,2
5,505,NORMAL,1,1
6,516,NORMAL,1,1
7,521,DEFINATE DELAY,1,1
8,521,NORMAL,1,1
9,552,DEFINATE DELAY,1,1


In [130]:
statement_trx_4 = '''SELECT distinct D.TMSTAMP, D.TRX_DATE, d.TRX_UNIT, d.TRX_USR, d.TRX_USR_SN, d.TUN_INTERNAL_SN, TRIM (TO_CHAR (D.TRX_DATE, 'yyyymmdd')) || '' || TRIM (D.TRX_UNIT) || '' || TRIM (D.TRX_USR) || '' || TRIM (ABS (D.TRX_USR_SN) - 1) || '' || TRIM (D.TUN_INTERNAL_SN) AS TRANSACTION_REF, P.DESCRIPTION AS PRODUCT_DESCRIPTION, R.ACCOUNT_NUMBER, P.ID_PRODUCT,A.FK_CUSTOMERCUST_ID,C.ID_NO as NATIONAL_ID,s.DATE_OF_BIRTH,s.CUST_OPEN_DATE, S.MOBILE_TEL, A.DESIGNATION AS ACCOUNT_NAME, D.I_ID_JUSTIFIC, J.DESCRIPTION AS TRANSACTION_TYPE, ABS (D.I_AMOUNT) "TRANSACTION_AMOUNT", D.ACC_AMOUNT_40 AS COMMISSION, D.I_COMMENTS TRX_COMMENTS, F.REVERSE_FLAG, F.REVERSED_TRX_FLAG, A.ACCOUNT_TYPE, (CASE WHEN A.ACCOUNT_TYPE = '1' THEN 'SINGLE' WHEN A.ACCOUNT_TYPE = '2' THEN 'JOINT' WHEN A.ACCOUNT_TYPE = '3' THEN 'COMMON'  END) AS ACCOUNT_TYPE_DECODE 
FROM prod.DEP_TRX_RECORDING D, prod.JUSTIFIC J, prod.PRODUCT P, prod.DEPOSIT_ACCOUNT A, prod.PROFITS_ACCOUNT R, prod.FST_DEMAND_EXTRAIT F, prod.OTHER_ID C, prod.CUSTOMER S
WHERE  A.ACCOUNT_NUMBER = D.I_ACCOUNT_NUMBER
AND R.DEP_ACC_NUMBER = D.I_ACCOUNT_NUMBER
AND R.DEP_ACC_NUMBER = F.FK_DEPOSIT_ACCOACC
AND A.ACCOUNT_NUMBER = F.FK_DEPOSIT_ACCOACC
AND D.ID_PRODUCT = P.ID_PRODUCT
AND D.I_ID_JUSTIFIC = J.ID_JUSTIFIC
AND F.TRX_DATE = D.TRX_DATE
AND F.TRX_UNIT = D.TRX_UNIT
AND C.FK_CUSTOMERCUST_ID=A.FK_CUSTOMERCUST_ID
AND S.CUST_ID=A.FK_CUSTOMERCUST_ID
AND F.TRX_SN = D.TRX_USR_SN
AND F.TRX_USR = D.TRX_USR
AND F.TUN_INTERNAL_SN = D.TUN_INTERNAL_SN
AND D.TRX_DATE BETWEEN TO_DATE ('01-01-2019', 'dd-mm-yyyy')
AND TO_DATE ('20-12-2019', 'dd-mm-yyyy')
--AND D.I_ID_JUSTIFIC IN (31102,31103,31104,31112,31113,31501,31511,31521,31531,31541,31551,31561,31572,31703,31704,31705,31718,31719,31721,31730,31731,31732,31750,31761,31771)
--AND D.I_ID_JUSTIFIC IN (31111,31705) #need to find the staff proucts
AND R.PRFT_SYSTEM = 3
AND F.REVERSE_FLAG = 0
AND F.REVERSED_TRX_FLAG = 0
AND D.ACC_AMOUNT_40 <> 0
'''
trxns_data_trx_4 = pd.read_sql_query(statement_trx_4, connection)
trxns_data_trx_4

,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,ID_PRODUCT,FK_CUSTOMERCUST_ID,NATIONAL_ID,DATE_OF_BIRTH,CUST_OPEN_DATE,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG,ACCOUNT_TYPE,ACCOUNT_TYPE_DECODE
0,2019-04-24 12:49:31,2019-04-24,100,ESB0001,2137,1,20190424100ESB000121361,LENGO ACCOUNT,2000066709,31103,100014,22215605,1981-04-10 00:00:00,2016-01-01 00:00:00,254720920166,MARTIN MUGO MARINGI AND OR RUTH W.WACHIR ...,31815,CR FROM MOBILE BANKING-MPESA TO ACC,70000.0,30.0,NDO7GD8MXP-254725717306 MPESATOACC,0,0,2,JOINT
1,2019-10-10 13:33:55,2019-10-11,100,ATMUSER,136,1,20191011100ATMUSER1351,LENGO ACCOUNT,2000066941,31103,100025,1005312,1959-10-31 00:00:00,2016-01-01 00:00:00,254721226609,PHYLLIS MUGERO MIRUNGU ...,33008,ATM WITHDR OFFUS MASTERCARD,40000.0,250.0,ATM DEBIT,0,0,1,SINGLE
2,2019-01-08 11:53:30,2019-01-08,100,ATMUSER,88,1,20190108100ATMUSER871,LENGO ACCOUNT,2000066941,31103,100025,1005312,1959-10-31 00:00:00,2016-01-01 00:00:00,254721226609,PHYLLIS MUGERO MIRUNGU ...,33008,ATM WITHDR OFFUS MASTERCARD,14000.0,250.0,ATM DEBIT,0,0,1,SINGLE
3,2019-07-25 13:21:26,2019-07-25,300,TMM1919,45,1,20190725300TMM1919441,TRANSACTIONAL ACCOUNT-CA,2000007820,31719,10003,2270393,1945-11-05 00:00:00,2016-01-01 00:00:00,254721246613,MBURA AGNES ...,31615,OTC CASH WITHDRAWAL,5000.0,100.0,AGNES,0,0,1,SINGLE
4,2019-09-27 10:29:28,2019-09-27,109,KOCE0001,5029,1,20190927109KOCE000150281,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),55000.0,60.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
5,2019-09-20 16:46:22,2019-09-20,109,KOCE0001,10066,1,20190920109KOCE0001100651,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),30000.0,60.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
6,2019-09-13 18:46:33,2019-09-13,109,KOCE0001,11503,1,20190913109KOCE0001115021,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),25000.0,60.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
7,2019-08-14 18:42:13,2019-08-14,109,KOCE0001,13015,1,20190814109KOCE0001130141,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),2500.0,40.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
8,2019-07-31 13:51:29,2019-07-31,109,KOCE0001,9245,1,20190731109KOCE000192441,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),1000.0,30.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
9,2019-07-30 11:08:26,2019-07-30,109,KOCE0003,182,1,20190730109KOCE00031811,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),2000.0,40.0,SEND TO MPESA 254720238537,0,0,1,SINGLE


In [133]:
trxns_data_trx_4 = trxns_data_trx_4.query('ACCOUNT_TYPE == "1"')

In [134]:
df1 = trxns_data_trx_4
df1

,TMSTAMP,TRX_DATE,TRX_UNIT,TRX_USR,TRX_USR_SN,TUN_INTERNAL_SN,TRANSACTION_REF,PRODUCT_DESCRIPTION,ACCOUNT_NUMBER,ID_PRODUCT,FK_CUSTOMERCUST_ID,NATIONAL_ID,DATE_OF_BIRTH,CUST_OPEN_DATE,MOBILE_TEL,ACCOUNT_NAME,I_ID_JUSTIFIC,TRANSACTION_TYPE,TRANSACTION_AMOUNT,COMMISSION,TRX_COMMENTS,REVERSE_FLAG,REVERSED_TRX_FLAG,ACCOUNT_TYPE,ACCOUNT_TYPE_DECODE
1,2019-10-10 13:33:55,2019-10-11,100,ATMUSER,136,1,20191011100ATMUSER1351,LENGO ACCOUNT,2000066941,31103,100025,1005312,1959-10-31 00:00:00,2016-01-01 00:00:00,254721226609,PHYLLIS MUGERO MIRUNGU ...,33008,ATM WITHDR OFFUS MASTERCARD,40000.0,250.0,ATM DEBIT,0,0,1,SINGLE
2,2019-01-08 11:53:30,2019-01-08,100,ATMUSER,88,1,20190108100ATMUSER871,LENGO ACCOUNT,2000066941,31103,100025,1005312,1959-10-31 00:00:00,2016-01-01 00:00:00,254721226609,PHYLLIS MUGERO MIRUNGU ...,33008,ATM WITHDR OFFUS MASTERCARD,14000.0,250.0,ATM DEBIT,0,0,1,SINGLE
3,2019-07-25 13:21:26,2019-07-25,300,TMM1919,45,1,20190725300TMM1919441,TRANSACTIONAL ACCOUNT-CA,2000007820,31719,10003,2270393,1945-11-05 00:00:00,2016-01-01 00:00:00,254721246613,MBURA AGNES ...,31615,OTC CASH WITHDRAWAL,5000.0,100.0,AGNES,0,0,1,SINGLE
4,2019-09-27 10:29:28,2019-09-27,109,KOCE0001,5029,1,20190927109KOCE000150281,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),55000.0,60.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
5,2019-09-20 16:46:22,2019-09-20,109,KOCE0001,10066,1,20190920109KOCE0001100651,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),30000.0,60.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
6,2019-09-13 18:46:33,2019-09-13,109,KOCE0001,11503,1,20190913109KOCE0001115021,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),25000.0,60.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
7,2019-08-14 18:42:13,2019-08-14,109,KOCE0001,13015,1,20190814109KOCE0001130141,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),2500.0,40.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
8,2019-07-31 13:51:29,2019-07-31,109,KOCE0001,9245,1,20190731109KOCE000192441,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),1000.0,30.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
9,2019-07-30 11:08:26,2019-07-30,109,KOCE0003,182,1,20190730109KOCE00031811,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),2000.0,40.0,SEND TO MPESA 254720238537,0,0,1,SINGLE
10,2019-07-29 18:58:11,2019-07-29,109,KOCE0001,19231,1,20190729109KOCE0001192301,TRANSACTIONAL ACCOUNT-CA,2000066974,31719,100063,22334918,1980-12-10 00:00:00,2016-01-01 00:00:00,254720238537,PERVINCE ONYANGO OGUTU ...,32099,ACCOUNT TO MPESA(B2C),3000.0,54.0,SEND TO MPESA 254720238537,0,0,1,SINGLE


In [135]:
#df1.ACCOUNT_NUMBER = df1.ACCOUNT_NUMBER.astype('int64')

account_100 = df1.query('ACCOUNT_NUMBER == "2000066709"')
account_100

In [136]:
#column names and type
df1.dtypes

TMSTAMP                datetime64[ns]
TRX_DATE               datetime64[ns]
TRX_UNIT                        int64
TRX_USR                        object
TRX_USR_SN                      int64
TUN_INTERNAL_SN                 int64
TRANSACTION_REF                object
PRODUCT_DESCRIPTION            object
ACCOUNT_NUMBER                 object
ID_PRODUCT                      int64
FK_CUSTOMERCUST_ID              int64
NATIONAL_ID                    object
DATE_OF_BIRTH                  object
CUST_OPEN_DATE                 object
MOBILE_TEL                     object
ACCOUNT_NAME                   object
I_ID_JUSTIFIC                   int64
TRANSACTION_TYPE               object
TRANSACTION_AMOUNT            float64
COMMISSION                    float64
TRX_COMMENTS                   object
REVERSE_FLAG                   object
REVERSED_TRX_FLAG              object
ACCOUNT_TYPE                   object
ACCOUNT_TYPE_DECODE            object
dtype: object

In [137]:
NOW = df1.TRX_DATE.max()
NOW

Timestamp('2019-10-26 00:00:00')

In [138]:
transaction_table = df1.groupby(['ACCOUNT_NUMBER','FK_CUSTOMERCUST_ID','NATIONAL_ID','PRODUCT_DESCRIPTION','ACCOUNT_NAME']).agg({'TRX_DATE': lambda x: (NOW - x.max()).days, 'DATE_OF_BIRTH': lambda x: (dt.datetime.now() - x.max()).days,'CUST_OPEN_DATE': lambda x: (dt.datetime.now() - x.max()).days,'TRANSACTION_REF': lambda x: len(x), 'COMMISSION': lambda x: x.sum(),'TRANSACTION_AMOUNT': lambda x: x.sum()})
transaction_table.rename(columns = {'TRX_DATE': 'recency_of_transaction', 
                                   'COMMISSION': 'monetary_value_of_transaction',
                                   'TRANSACTION_AMOUNT': 'sum_Amount_of_transaction',
                                   'CUST_OPEN_DATE':'Age_cust_in_bank',
                                   'DATE_OF_BIRTH':'Age_customer',
                                   'TRANSACTION_REF':'count_of_transactions'})




,,,,,sum_Amount_of_transaction,monetary_value_of_transaction,recency_of_transaction,Age_cust_in_bank,Age_customer,count_of_transactions
ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,,,,,,
0000430004,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-HEAD OFFICE,0.0,1000.00,236,2239,737360,1
0000430015,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NAKURU,0.0,1000.00,47,2239,737360,1
0000430018,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NYERI,0.0,1000.00,171,2239,737360,1
0002963201,296,30479340,TRANSACTIONAL ACCOUNT-CA,ANNASTASIA KYALO,0.0,200.00,119,1370,9693,1
0002993201,299,32059758,TRANSACTIONAL ACCOUNT-CA,DEBORAH MWANU,0.0,200.00,119,1370,9704,1
0011663201,1166,3962085,MICRO BIZ ACCOUNT,JAYNE MUGECHI MOTIGA,89000.0,460.00,96,1397,22581,7
0012063201,1206,4827199,PERSONAL CURRENT ACCOUNT-UNBUN,WAIREGI WILFRED,25451.0,3510.00,26,1397,23490,12
0015443201,1544,192917,SME BIZ ACCOUNT,MWACHARO AND ASSOCIATES ARCHITECTS,0.0,2700.00,26,1397,737360,9
0016513201,1651,3103381,PERSONAL CURRENT ACCOUNT-UNBUN,MICHAEL MUCHAI KAGIKO,0.0,2700.00,26,1397,25503,9


In [139]:
transaction_table.reset_index(inplace= True)

In [140]:

transaction_table

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF
0,0000430004,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,0.0,1000.00,236,2239,737360,1
1,0000430015,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NAKURU ...,0.0,1000.00,47,2239,737360,1
2,0000430018,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NYERI ...,0.0,1000.00,171,2239,737360,1
3,0002963201,296,30479340,TRANSACTIONAL ACCOUNT-CA,ANNASTASIA KYALO ...,0.0,200.00,119,1370,9693,1
4,0002993201,299,32059758,TRANSACTIONAL ACCOUNT-CA,DEBORAH MWANU ...,0.0,200.00,119,1370,9704,1
5,0011663201,1166,3962085,MICRO BIZ ACCOUNT,JAYNE MUGECHI MOTIGA ...,89000.0,460.00,96,1397,22581,7
6,0012063201,1206,4827199,PERSONAL CURRENT ACCOUNT-UNBUN,WAIREGI WILFRED ...,25451.0,3510.00,26,1397,23490,12
7,0015443201,1544,192917,SME BIZ ACCOUNT,MWACHARO AND ASSOCIATES ARCHITECTS ...,0.0,2700.00,26,1397,737360,9
8,0016513201,1651,3103381,PERSONAL CURRENT ACCOUNT-UNBUN,MICHAEL MUCHAI KAGIKO ...,0.0,2700.00,26,1397,25503,9
9,0017773201,1777,1888478,TRANSACTIONAL ACCOUNT-CA,MARY WANJIKU WARUI ...,0.0,500.00,185,1397,25868,1


In [141]:
loans_table.dtypes

CUST_ID                          int64
LOAN_STATUS_DECODE              object
count_loan_products              int64
count_of_loans_taken_in_bank     int64
dtype: object

In [142]:
loans_table.head()

,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank
0,298,NORMAL,1,1
1,303,NORMAL,2,2
2,305,DEFINATE DELAY,1,1
3,314,NORMAL,1,1
4,505,DEFINATE DELAY,2,2


In [143]:
transaction_table.head()

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF
0,0000430004,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,0.0,1000.0,236,2239,737360,1
1,0000430015,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NAKURU ...,0.0,1000.0,47,2239,737360,1
2,0000430018,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NYERI ...,0.0,1000.0,171,2239,737360,1
3,0002963201,296,30479340,TRANSACTIONAL ACCOUNT-CA,ANNASTASIA KYALO ...,0.0,200.0,119,1370,9693,1
4,0002993201,299,32059758,TRANSACTIONAL ACCOUNT-CA,DEBORAH MWANU ...,0.0,200.0,119,1370,9704,1


In [144]:
all_cust_data = transaction_table.merge(loans_table, left_on='FK_CUSTOMERCUST_ID', right_on='CUST_ID', how = 'left')

In [145]:
#reference https://towardsdatascience.com/find-your-best-customers-with-customer-segmentation-in-python-61d602f9eee6
all_cust_data_1 = all_cust_data

In [146]:
all_cust_data

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank
0,0000430004,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,0.0,1000.00,236,2239,737360,1,NaN,NaN,NaN,NaN
1,0000430015,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NAKURU ...,0.0,1000.00,47,2239,737360,1,NaN,NaN,NaN,NaN
2,0000430018,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NYERI ...,0.0,1000.00,171,2239,737360,1,NaN,NaN,NaN,NaN
3,0002963201,296,30479340,TRANSACTIONAL ACCOUNT-CA,ANNASTASIA KYALO ...,0.0,200.00,119,1370,9693,1,NaN,NaN,NaN,NaN
4,0002993201,299,32059758,TRANSACTIONAL ACCOUNT-CA,DEBORAH MWANU ...,0.0,200.00,119,1370,9704,1,NaN,NaN,NaN,NaN
5,0011663201,1166,3962085,MICRO BIZ ACCOUNT,JAYNE MUGECHI MOTIGA ...,89000.0,460.00,96,1397,22581,7,NaN,NaN,NaN,NaN
6,0012063201,1206,4827199,PERSONAL CURRENT ACCOUNT-UNBUN,WAIREGI WILFRED ...,25451.0,3510.00,26,1397,23490,12,1206.0,NORMAL,1.0,1.0
7,0015443201,1544,192917,SME BIZ ACCOUNT,MWACHARO AND ASSOCIATES ARCHITECTS ...,0.0,2700.00,26,1397,737360,9,NaN,NaN,NaN,NaN
8,0016513201,1651,3103381,PERSONAL CURRENT ACCOUNT-UNBUN,MICHAEL MUCHAI KAGIKO ...,0.0,2700.00,26,1397,25503,9,1651.0,NORMAL,1.0,1.0
9,0017773201,1777,1888478,TRANSACTIONAL ACCOUNT-CA,MARY WANJIKU WARUI ...,0.0,500.00,185,1397,25868,1,NaN,NaN,NaN,NaN


In [147]:
all_cust_data.isna().mean().round(4) * 100

ACCOUNT_NUMBER                   0.00
FK_CUSTOMERCUST_ID               0.00
NATIONAL_ID                      0.00
PRODUCT_DESCRIPTION              0.00
ACCOUNT_NAME                     0.00
TRANSACTION_AMOUNT               0.00
COMMISSION                       0.00
TRX_DATE                         0.00
CUST_OPEN_DATE                   0.00
DATE_OF_BIRTH                    0.00
TRANSACTION_REF                  0.00
CUST_ID                         47.97
LOAN_STATUS_DECODE              47.97
count_loan_products             47.97
count_of_loans_taken_in_bank    47.97
dtype: float64

In [148]:
all_cust_data.count()

ACCOUNT_NUMBER                  131051
FK_CUSTOMERCUST_ID              131051
NATIONAL_ID                     131051
PRODUCT_DESCRIPTION             131051
ACCOUNT_NAME                    131051
TRANSACTION_AMOUNT              131051
COMMISSION                      131051
TRX_DATE                        131051
CUST_OPEN_DATE                  131051
DATE_OF_BIRTH                   131051
TRANSACTION_REF                 131051
CUST_ID                          68191
LOAN_STATUS_DECODE               68191
count_loan_products              68191
count_of_loans_taken_in_bank     68191
dtype: int64

In [149]:
def ScoreCountLoans(x):
    if x ==0:
        return -100
    elif x <=1:
        return 10
    elif x <=6:
        return 20
    elif x<=7:
        return 60
    else:
        return 0

def CountOfDefaults(x):
    if x == 0:
        return 0
    elif x <=1:
        return -10
    elif x<=2:
        return -30
    elif x<=6:
        return -50
    elif x>=7:
        return -100
    else:
        return 0

In [150]:

def Sum_of_transact(x):
    if x<= 100:
        return -50
    elif x<= 3000:
        return 10
    elif x<= 5000:
        return 15
    elif x>=5000:
        return 30
    else:
        return 0

def age_cust_bank(x):
    if x<=30:
        return -5
    elif x<=90:
        return 1
    elif x<=180:
        return 2
    elif x<=360:
        return 10
    elif x>=361:
        return 100
    else:
        return 0
#CASE WHEN a.ACC_STATUS = '1' THEN 'NORMAL' WHEN a.ACC_STATUS = '2' THEN 'OVERDUE' WHEN a.ACC_STATUS = '3' THEN 'DEFINATE DELAY' WHEN a.ACC_STATUS = '4' THEN 'WRITE OFF'
def has_delayed_loan(x):
    if x=='NORMAL':
        return 0
    elif x == 'OVERDUE':
        return -100
    elif x=='DEFINATE DELAY':
        return -150
    elif x=='WRITE OFF':
        return -200
    else:
        return 0    

In [151]:
all_cust_data

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank
0,0000430004,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-HEAD OFFICE ...,0.0,1000.00,236,2239,737360,1,NaN,NaN,NaN,NaN
1,0000430015,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NAKURU ...,0.0,1000.00,47,2239,737360,1,NaN,NaN,NaN,NaN
2,0000430018,43,00000,MEMO PRODUCT-JOURNAL,SALARY SUSPENSE ACCOUNT-NYERI ...,0.0,1000.00,171,2239,737360,1,NaN,NaN,NaN,NaN
3,0002963201,296,30479340,TRANSACTIONAL ACCOUNT-CA,ANNASTASIA KYALO ...,0.0,200.00,119,1370,9693,1,NaN,NaN,NaN,NaN
4,0002993201,299,32059758,TRANSACTIONAL ACCOUNT-CA,DEBORAH MWANU ...,0.0,200.00,119,1370,9704,1,NaN,NaN,NaN,NaN
5,0011663201,1166,3962085,MICRO BIZ ACCOUNT,JAYNE MUGECHI MOTIGA ...,89000.0,460.00,96,1397,22581,7,NaN,NaN,NaN,NaN
6,0012063201,1206,4827199,PERSONAL CURRENT ACCOUNT-UNBUN,WAIREGI WILFRED ...,25451.0,3510.00,26,1397,23490,12,1206.0,NORMAL,1.0,1.0
7,0015443201,1544,192917,SME BIZ ACCOUNT,MWACHARO AND ASSOCIATES ARCHITECTS ...,0.0,2700.00,26,1397,737360,9,NaN,NaN,NaN,NaN
8,0016513201,1651,3103381,PERSONAL CURRENT ACCOUNT-UNBUN,MICHAEL MUCHAI KAGIKO ...,0.0,2700.00,26,1397,25503,9,1651.0,NORMAL,1.0,1.0
9,0017773201,1777,1888478,TRANSACTIONAL ACCOUNT-CA,MARY WANJIKU WARUI ...,0.0,500.00,185,1397,25868,1,NaN,NaN,NaN,NaN


In [152]:
all_cust_data['score_count_of_loans_taken_in_bank'] = all_cust_data['count_of_loans_taken_in_bank'].apply(ScoreCountLoans)


all_cust_data['score_sum_Amount_of_transaction'] = all_cust_data['TRANSACTION_AMOUNT'].apply(Sum_of_transact)


all_cust_data['score_age_of_customer'] = all_cust_data['CUST_OPEN_DATE'].apply(age_cust_bank)


all_cust_data['score_LOAN_STATUS_DECODE'] = all_cust_data['LOAN_STATUS_DECODE'].apply(has_delayed_loan)


all_cust_data['Total_score_hf'] = all_cust_data['score_count_of_loans_taken_in_bank'] + all_cust_data['score_sum_Amount_of_transaction'] + all_cust_data['score_age_of_customer']+all_cust_data['score_LOAN_STATUS_DECODE']



In [153]:
lendable_cust = all_cust_data[all_cust_data.Total_score_hf>0]

In [154]:
lendable_cust.count()

ACCOUNT_NUMBER                        78515
FK_CUSTOMERCUST_ID                    78515
NATIONAL_ID                           78515
PRODUCT_DESCRIPTION                   78515
ACCOUNT_NAME                          78515
TRANSACTION_AMOUNT                    78515
COMMISSION                            78515
TRX_DATE                              78515
CUST_OPEN_DATE                        78515
DATE_OF_BIRTH                         78515
TRANSACTION_REF                       78515
CUST_ID                               24625
LOAN_STATUS_DECODE                    24625
count_loan_products                   24625
count_of_loans_taken_in_bank          24625
score_count_of_loans_taken_in_bank    78515
score_sum_Amount_of_transaction       78515
score_age_of_customer                 78515
score_LOAN_STATUS_DECODE              78515
Total_score_hf                        78515
dtype: int64

In [155]:
lendable_cust.PRODUCT_DESCRIPTION.unique()

array(['MEMO PRODUCT-JOURNAL', 'TRANSACTIONAL ACCOUNT-CA',
       'MICRO BIZ ACCOUNT', 'PERSONAL CURRENT ACCOUNT-UNBUN',
       'SME BIZ ACCOUNT', 'CROSS OVER SAVING ACCOUNT', 'LENGO ACCOUNT',
       'HFC SALARY ACCOUNT', 'CURRENT ACCOUNT CORPORATE',
       'PERSONAL CURRENT ACCOUNT BUND', 'CURRENT USD',
       'HFC KILIMO ACCOUNT', 'SAVINGS USD', 'YOLO ACCOUNT', '1ST HOP',
       'TAKE ON ACCOUNT', 'SAVINGS EURO', 'STAFF SALARY ACCOUNT',
       'KUNDI BORA ACCOUNT', 'CURRENT ACCOUNT STAFF',
       'EASY COLLECT ACCOUNT', 'CURRENT EURO', 'CURRENT GBP',
       'SAVINGS GBP', 'ESCROW CURRENT ACCOUNT', 'APEX ACCOUNT',
       'FLEXI ACCOUNT', 'TREASURE SAVINGS ACCOUNT',
       'SUBSCRIPTION BOND SAVINGS ACCO', 'REGISTRAR HIGH COURT SAVINGS A',
       'MAKAO SAVING ACCOUNT', 'STAFF SAVINGS ACCOUNT',
       'VIRTUAL ACCOUNT MOBILE', 'INSTITUTIONAL BANKING-CA'], dtype=object)

In [156]:
lendable_cust_1 = lendable_cust.query('PRODUCT_DESCRIPTION in ["TRANSACTIONAL ACCOUNT-CA", "PERSONAL CURRENT ACCOUNT-UNBUN","MICRO BIZ ACCOUNT","SME BIZ ACCOUNT","LENGO ACCOUNT","HFC SALARY ACCOUNT","PERSONAL CURRENT ACCOUNT BUND","HFC KILIMO ACCOUNT","EASY COLLECT ACCOUNT","YOLO ACCOUNT","VIRTUAL ACCOUNT MOBILE"]') #"MICRO BIZ ACCOUNT",



In [157]:
lendable_cust_1 

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank,score_count_of_loans_taken_in_bank,score_sum_Amount_of_transaction,score_age_of_customer,score_LOAN_STATUS_DECODE,Total_score_hf
3,0002963201,296,30479340,TRANSACTIONAL ACCOUNT-CA,ANNASTASIA KYALO ...,0.0,200.0,119,1370,9693,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
4,0002993201,299,32059758,TRANSACTIONAL ACCOUNT-CA,DEBORAH MWANU ...,0.0,200.0,119,1370,9704,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
5,0011663201,1166,3962085,MICRO BIZ ACCOUNT,JAYNE MUGECHI MOTIGA ...,89000.0,460.0,96,1397,22581,7,NaN,NaN,NaN,NaN,0,30,100,0,130
6,0012063201,1206,4827199,PERSONAL CURRENT ACCOUNT-UNBUN,WAIREGI WILFRED ...,25451.0,3510.0,26,1397,23490,12,1206.0,NORMAL,1.0,1.0,10,30,100,0,140
7,0015443201,1544,192917,SME BIZ ACCOUNT,MWACHARO AND ASSOCIATES ARCHITECTS ...,0.0,2700.0,26,1397,737360,9,NaN,NaN,NaN,NaN,0,-50,100,0,50
8,0016513201,1651,3103381,PERSONAL CURRENT ACCOUNT-UNBUN,MICHAEL MUCHAI KAGIKO ...,0.0,2700.0,26,1397,25503,9,1651.0,NORMAL,1.0,1.0,10,-50,100,0,60
9,0017773201,1777,1888478,TRANSACTIONAL ACCOUNT-CA,MARY WANJIKU WARUI ...,0.0,500.0,185,1397,25868,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
11,0017933202,1793,3110009,PERSONAL CURRENT ACCOUNT-UNBUN,PETER MURIMI NGWACI ...,439000.0,2900.0,26,1397,21713,10,1793.0,NORMAL,2.0,2.0,20,30,100,0,150
13,0021313201,2131,11515896,MICRO BIZ ACCOUNT,PATRICK MUGO WANJOHI ...,23100.0,620.0,31,1397,17749,4,NaN,NaN,NaN,NaN,0,30,100,0,130
14,0021803201,2180,1902865,MICRO BIZ ACCOUNT,MERCY WANGU MWANGI ...,137000.0,700.0,40,1397,26964,3,NaN,NaN,NaN,NaN,0,30,100,0,130


In [158]:
lendable_cust_2 = lendable_cust_1[lendable_cust_1.Total_score_hf>=50]

In [159]:
lendable_cust_2

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank,score_count_of_loans_taken_in_bank,score_sum_Amount_of_transaction,score_age_of_customer,score_LOAN_STATUS_DECODE,Total_score_hf
3,0002963201,296,30479340,TRANSACTIONAL ACCOUNT-CA,ANNASTASIA KYALO ...,0.0,200.0,119,1370,9693,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
4,0002993201,299,32059758,TRANSACTIONAL ACCOUNT-CA,DEBORAH MWANU ...,0.0,200.0,119,1370,9704,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
5,0011663201,1166,3962085,MICRO BIZ ACCOUNT,JAYNE MUGECHI MOTIGA ...,89000.0,460.0,96,1397,22581,7,NaN,NaN,NaN,NaN,0,30,100,0,130
6,0012063201,1206,4827199,PERSONAL CURRENT ACCOUNT-UNBUN,WAIREGI WILFRED ...,25451.0,3510.0,26,1397,23490,12,1206.0,NORMAL,1.0,1.0,10,30,100,0,140
7,0015443201,1544,192917,SME BIZ ACCOUNT,MWACHARO AND ASSOCIATES ARCHITECTS ...,0.0,2700.0,26,1397,737360,9,NaN,NaN,NaN,NaN,0,-50,100,0,50
8,0016513201,1651,3103381,PERSONAL CURRENT ACCOUNT-UNBUN,MICHAEL MUCHAI KAGIKO ...,0.0,2700.0,26,1397,25503,9,1651.0,NORMAL,1.0,1.0,10,-50,100,0,60
9,0017773201,1777,1888478,TRANSACTIONAL ACCOUNT-CA,MARY WANJIKU WARUI ...,0.0,500.0,185,1397,25868,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
11,0017933202,1793,3110009,PERSONAL CURRENT ACCOUNT-UNBUN,PETER MURIMI NGWACI ...,439000.0,2900.0,26,1397,21713,10,1793.0,NORMAL,2.0,2.0,20,30,100,0,150
13,0021313201,2131,11515896,MICRO BIZ ACCOUNT,PATRICK MUGO WANJOHI ...,23100.0,620.0,31,1397,17749,4,NaN,NaN,NaN,NaN,0,30,100,0,130
14,0021803201,2180,1902865,MICRO BIZ ACCOUNT,MERCY WANGU MWANGI ...,137000.0,700.0,40,1397,26964,3,NaN,NaN,NaN,NaN,0,30,100,0,130


In [160]:
#export_csv = all_cust_data.to_csv (r'/Users/allan.aswani/Documents/scorecardworks/all_cust_data_repeat_score_card.csv', header=True) #Don't forget to add '.csv' at the end of the path
#customers_report = pd.read_excel(r'/Users/allan.aswani/Downloads/calculation/All_whiz_Active_customers.xlsx')


In [165]:
lendable_cust_3 =lendable_cust_2[~lendable_cust_2.NATIONAL_ID.str[19].str.isdigit()]
lendable_cust_3

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank,score_count_of_loans_taken_in_bank,score_sum_Amount_of_transaction,score_age_of_customer,score_LOAN_STATUS_DECODE,Total_score_hf
3,0002963201,296,30479340,TRANSACTIONAL ACCOUNT-CA,ANNASTASIA KYALO ...,0.0,200.0,119,1370,9693,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
4,0002993201,299,32059758,TRANSACTIONAL ACCOUNT-CA,DEBORAH MWANU ...,0.0,200.0,119,1370,9704,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
5,0011663201,1166,3962085,MICRO BIZ ACCOUNT,JAYNE MUGECHI MOTIGA ...,89000.0,460.0,96,1397,22581,7,NaN,NaN,NaN,NaN,0,30,100,0,130
6,0012063201,1206,4827199,PERSONAL CURRENT ACCOUNT-UNBUN,WAIREGI WILFRED ...,25451.0,3510.0,26,1397,23490,12,1206.0,NORMAL,1.0,1.0,10,30,100,0,140
7,0015443201,1544,192917,SME BIZ ACCOUNT,MWACHARO AND ASSOCIATES ARCHITECTS ...,0.0,2700.0,26,1397,737360,9,NaN,NaN,NaN,NaN,0,-50,100,0,50
8,0016513201,1651,3103381,PERSONAL CURRENT ACCOUNT-UNBUN,MICHAEL MUCHAI KAGIKO ...,0.0,2700.0,26,1397,25503,9,1651.0,NORMAL,1.0,1.0,10,-50,100,0,60
9,0017773201,1777,1888478,TRANSACTIONAL ACCOUNT-CA,MARY WANJIKU WARUI ...,0.0,500.0,185,1397,25868,1,NaN,NaN,NaN,NaN,0,-50,100,0,50
11,0017933202,1793,3110009,PERSONAL CURRENT ACCOUNT-UNBUN,PETER MURIMI NGWACI ...,439000.0,2900.0,26,1397,21713,10,1793.0,NORMAL,2.0,2.0,20,30,100,0,150
13,0021313201,2131,11515896,MICRO BIZ ACCOUNT,PATRICK MUGO WANJOHI ...,23100.0,620.0,31,1397,17749,4,NaN,NaN,NaN,NaN,0,30,100,0,130
14,0021803201,2180,1902865,MICRO BIZ ACCOUNT,MERCY WANGU MWANGI ...,137000.0,700.0,40,1397,26964,3,NaN,NaN,NaN,NaN,0,30,100,0,130


In [162]:
#export_csv = lendable_cust_2.to_excel (r'/Users/allan.aswani/Documents/scorecardworks/all_cust_data_repeat_score_card_data_tu_version', header=True) #Don't forget to add '.csv' at the end of the path

lendable_cust_3.to_csv(r'/Users/allan.aswani/Documents/scorecardworks/all_cust_data_repeat_score_card_data_tu_version.csv', header=True)

In [181]:
lendable_cust_3.dtypes

ACCOUNT_NUMBER                         object
FK_CUSTOMERCUST_ID                      int64
NATIONAL_ID                            object
PRODUCT_DESCRIPTION                    object
ACCOUNT_NAME                           object
TRANSACTION_AMOUNT                    float64
COMMISSION                            float64
TRX_DATE                                int64
CUST_OPEN_DATE                          int64
DATE_OF_BIRTH                           int64
TRANSACTION_REF                         int64
CUST_ID                               float64
LOAN_STATUS_DECODE                     object
count_loan_products                   float64
count_of_loans_taken_in_bank          float64
score_count_of_loans_taken_in_bank      int64
score_sum_Amount_of_transaction         int64
score_age_of_customer                   int64
score_LOAN_STATUS_DECODE                int64
Total_score_hf                          int64
dtype: object

In [280]:
tu_data = pd.read_excel(r'/Users/allan.aswani/Documents/scorecardworks/HF_repeat_customer_for_transunion_manual_scoring.xlsx',sheet_name  = 'account_summary')

In [281]:
#tu_data = pd.DataFrame(tu_data)

In [282]:
tu_data.NATIONALID

0         5176836
1         5197891
2         5334407
3         5379876
4         5431661
5         5472930
6         7763285
7         7812443
8         7878777
9         7947664
10         900314
11         920312
12         935992
13         954744
14         964702
15         973637
16         977310
17         985970
18         987389
19       36773777
20       36909012
21       37007178
22        3317387
23        3329051
24        3338874
25        3348144
26        3357635
27        3392346
28       23953008
29        2128962
           ...   
30853    25661164
30854    25680275
30855    25703047
30856    26147297
30857    26167427
30858    26190594
30859    26207745
30860    26235255
30861    26254301
30862    26267457
30863    26276341
30864    26298288
30865    26309231
30866    26333800
30867    26346623
30868    26358369
30869    26381033
30870    26400103
30871    27073396
30872    27082233
30873    27086061
30874    27093301
30875    27101718
30876    27102902
30877    2

In [283]:
lendable_cust_3.merge(tu_data, left_on='NATIONAL_ID', right_on= 'NATIONALID', how= 'inner')

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank,score_count_of_loans_taken_in_bank,score_sum_Amount_of_transaction,score_age_of_customer,score_LOAN_STATUS_DECODE,Total_score_hf,CRN,RETRODATE,FIRSTNAME,SURNAME,NATIONALID,PASSPORTNO,MOBILENO,SERVICEID,ALIENID,DATEOFBIRTH,REGISTRATIONNUMBER,REGISTRATIONDATE,EMPLOYER,EMPLOYMENTPERIOD,INCOME,INCOMEFREQUENCY,HOUSETYPE,PHYSICALADDRESS,NUMBEROFPHYSICALADDRESSES,PHYSICALADDRESSLASTUPDATEDATE,DURATIONATPHYSICALADDRESS,NUMBEROFPOSTALADDRESSES,POSTALADDRESSLASTUPDATEDATE,NUMBEROFPHONECONTACTS,PHONECONTACTSLASTUPDATEDATE,FIRSTLISTEDDATE,NUMBEROF_ACCOUNTS_,INSTALMENTACCOUNT,REVOLVINGACCOUNT,OPENACCOUNT,CREDITCARD,HOMELOAN,SINGLECREDITFACILITY,OVERDRAFT,PENSIONBACKED,STUDENTLOAN,COLLECTIONACCOUNT,LINEOFCREDIT,PERFORMING,NONPERFORMING,CLOSED,CURRENTINARREARS,MAXARREARS,ARREARS0DAYS,ARREARS30DAYS,ARREARS60DAYS,ARREARS90DAYS,ARREARSGT90DAYS,EARLIESTDATEOPENED,PRINCIPALAMOUNT,SUBSCRIBERPRINCIPALAMOUNT,SCHEDULEDPAYMENTAMT,COLLATERALSGIVEN,CREDITAPPLICATIONSDONE,FRAUDCOUNT,CURRENTBALANCEAMOUNT,SUBSCRIBERCURRENTBALANCEAMOUNT,PASTDUEAMOUNT,LASTPAYMENTDATE,X_ACCOUNTS_DISPUTED,NOOFCHEQUESRETURNED,EARLIESTCHEQUERETURNDATE,BOUNCEDCHEQUEAMOUNT,NODDACS,EARLIESTDDACBOUNCEDATE,DDACBOUNCEDAMOUNT,NUMBEROFENQUIRIES,LASTENQUIRYDATE,ENQUIRIES30DAYS,ENQUIRIES60DAYS,ENQUIRIES90DAYS,ENQUIRIESGT90DAYS,SCORE,SCOREGRADE,PROBABILITY,SCORE_MB,SCOREGRADE_MB,PROBABILITY_MB,CONSUMERTYPE,CLIENTID,AARC1,AARC2,AARC3,AARC4,AARCM1,AARCM2,AARCM3,AARCM4,MOBILE_TOTAL,MOBILE_PERFORMING,MOBILE_NONPERFORMING,MOBILE_CLOSED,MOBILE_CURRENTINARREARS,MOBILE_PRINCIPALAMOUNT,MOBILE_SUBSCRIBERPRINCIPAL,MOBILE_CURRENTBALANCEAMOUNT,MOBILE_SUBSCRIBERCURRENTBALANCEAMOUNT,MOBILE_LASTPAYMENTDATE


In [284]:
lendable_cust_3.query('NATIONAL_ID =="5176836"' )

,ACCOUNT_NUMBER,FK_CUSTOMERCUST_ID,NATIONAL_ID,PRODUCT_DESCRIPTION,ACCOUNT_NAME,TRANSACTION_AMOUNT,COMMISSION,TRX_DATE,CUST_OPEN_DATE,DATE_OF_BIRTH,TRANSACTION_REF,CUST_ID,LOAN_STATUS_DECODE,count_loan_products,count_of_loans_taken_in_bank,score_count_of_loans_taken_in_bank,score_sum_Amount_of_transaction,score_age_of_customer,score_LOAN_STATUS_DECODE,Total_score_hf


In [285]:
tu_data

,CRN,RETRODATE,FIRSTNAME,SURNAME,NATIONALID,PASSPORTNO,MOBILENO,SERVICEID,ALIENID,DATEOFBIRTH,REGISTRATIONNUMBER,REGISTRATIONDATE,EMPLOYER,EMPLOYMENTPERIOD,INCOME,INCOMEFREQUENCY,HOUSETYPE,PHYSICALADDRESS,NUMBEROFPHYSICALADDRESSES,PHYSICALADDRESSLASTUPDATEDATE,DURATIONATPHYSICALADDRESS,NUMBEROFPOSTALADDRESSES,POSTALADDRESSLASTUPDATEDATE,NUMBEROFPHONECONTACTS,PHONECONTACTSLASTUPDATEDATE,FIRSTLISTEDDATE,NUMBEROF_ACCOUNTS_,INSTALMENTACCOUNT,REVOLVINGACCOUNT,OPENACCOUNT,CREDITCARD,HOMELOAN,SINGLECREDITFACILITY,OVERDRAFT,PENSIONBACKED,STUDENTLOAN,COLLECTIONACCOUNT,LINEOFCREDIT,PERFORMING,NONPERFORMING,CLOSED,CURRENTINARREARS,MAXARREARS,ARREARS0DAYS,ARREARS30DAYS,ARREARS60DAYS,ARREARS90DAYS,ARREARSGT90DAYS,EARLIESTDATEOPENED,PRINCIPALAMOUNT,SUBSCRIBERPRINCIPALAMOUNT,SCHEDULEDPAYMENTAMT,COLLATERALSGIVEN,CREDITAPPLICATIONSDONE,FRAUDCOUNT,CURRENTBALANCEAMOUNT,SUBSCRIBERCURRENTBALANCEAMOUNT,PASTDUEAMOUNT,LASTPAYMENTDATE,X_ACCOUNTS_DISPUTED,NOOFCHEQUESRETURNED,EARLIESTCHEQUERETURNDATE,BOUNCEDCHEQUEAMOUNT,NODDACS,EARLIESTDDACBOUNCEDATE,DDACBOUNCEDAMOUNT,NUMBEROFENQUIRIES,LASTENQUIRYDATE,ENQUIRIES30DAYS,ENQUIRIES60DAYS,ENQUIRIES90DAYS,ENQUIRIESGT90DAYS,SCORE,SCOREGRADE,PROBABILITY,SCORE_MB,SCOREGRADE_MB,PROBABILITY_MB,CONSUMERTYPE,CLIENTID,AARC1,AARC2,AARC3,AARC4,AARCM1,AARCM2,AARCM3,AARCM4,MOBILE_TOTAL,MOBILE_PERFORMING,MOBILE_NONPERFORMING,MOBILE_CLOSED,MOBILE_CURRENTINARREARS,MOBILE_PRINCIPALAMOUNT,MOBILE_SUBSCRIBERPRINCIPAL,MOBILE_CURRENTBALANCEAMOUNT,MOBILE_SUBSCRIBERCURRENTBALANCEAMOUNT,MOBILE_LASTPAYMENTDATE
0,7574778,30-10-2019,BERNARD WAMICHI,KIMUHU,5176836,NaN,NaN,NaN,NaN,1951-11-11 00:00:00,NaN,NaN,NIC BANK,296.0,0.0,NaN,NaN,P.O BOX 51324,4.0,13-Mar-2019,0.0,8.0,13-Mar-2019,3.0,NaN,12-05-2014,12.0,0.0,0.0,5.0,3.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,12.0,0.0,7.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,NaN,9.362806e+06,5500343.0,1.375656e+05,5.0,1.0,0.0,3916462.71,3596088.63,0.00,11-10-2019,NaN,0.0,NaN,0.0,0.0,NaN,0.0,8.0,26-07-2017,0.0,0.0,0.0,8.0,712,AA,5.03,663.0,BB,13.54,INDIVIDUAL,NaN,Many loan applications in the recent past (for...,"High number of open, longterm loans where the...",Many unpaid installments or no loans that have...,Loan installments haven't been paid in the rec...,Low number of payments recently,Satisfactory accounts are relatively old,Recent delinquency on accounts,Average credit limit of Revolving accounts are...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,NaN
1,3338743,30-10-2019,HANNA WAMBUI,MBIRIRI,5197891,NaN,NaN,NaN,NaN,1931-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,09-Nov-2010,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.00,0.00,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,YY,999.99,0.0,YY,999.99,INDIVIDUAL,NaN,-,-,-,-,-,-,-,-,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,NaN
2,7622950,30-10-2019,MUCHAI IHUTHIA,MURIU,5334407,NaN,NaN,NaN,NaN,1949-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,14-Sep-2019,0.0,3.0,15-Mar-2019,2.0,NaN,18-03-2017,36.0,0.0,0.0,19.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,36.0,0.0,17.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,NaN,3.727327e+05,0.0,7.129940e+04,10.0,10.0,0.0,43877.65,0.00,0.00,24-10-2019,NaN,0.0,NaN,0.0,0.0,NaN,0.0,3.0,21-06-2017,0.0,0.0,0.0,3.0,517,II,44.25,695.0,AA,9.37,INDIVIDUAL,NaN,Average credit limit for loans is relatively low,Borrowed a lot on loans other than secured loa...,Many loan applications in the recent past (for...,Many unpaid installments or no loans that have...,Average credit limit of accounts are relativel...,Low number of payments recently,Satisfactory accounts are relatively old,Recent delinquency on accounts,14.0,14.0,0.0,3.0,0.0,63828.35,0.0,22028.350,0.0,2019-05-20
3,328298,30-10-2019,BEATRICE WANGARI,MWANGI,5379876,NaN,NaN,NaN,NaN,1962-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P O BOX 84883- 80100 MOMBASA,2.0,06-Mar-2019,0.0,3.0,16-Mar-2017,3.0,NaN,07-08-2014,2.0,0.0,0.0,1.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,2.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0

In [286]:
def tu_NONPERFORMING(x):
    if x <1:
        return 140
    elif x<=3:
        return -14
    elif x<=5:
        return -108
    elif x<5:
        return -153
    else:
        return 0


def tu_ARREARS0DAYS(x):
    if x<1:
        return 0
    elif x <=10:
        return -75
    elif x<=30:
        return -20
    elif x<=60:
        return 14
    elif x<=75:
        return 43
    elif x<75:
        return 74
    else:
        return 0

def tu_PROBABILITY (x):
    if x<=20:
        return 102
    elif x<=26:
        return 59
    elif x<=76:
        return 3
    elif x<76:
        return -34
    else:
        return 0
def tu_SCORE_MB (x):
    if x<=480:
        return -39
    elif x<=580:
        return -15
    elif x<=640:
        return 13
    elif x<=680:
        return 56
    elif x< 680:
        return 114
    else:
        return 0

def tu_PASTDUEAMOUNT (x):
    if x<1:
        return 0
    
    elif x <= 2000:
        return -42
    elif x<= -20000:
        return -12
    elif x< 20000:
        return -19
    else:
        return 0
    
    

In [287]:
tu_data['score_tu_NONPERFORMING'] = tu_data.NONPERFORMING.apply(tu_NONPERFORMING)
tu_data['score_tu_ARREARS0DAYS'] = tu_data['ARREARS0DAYS'].apply(tu_ARREARS0DAYS)
tu_data['score_tu_PROBABILITY'] = tu_data.PROBABILITY.apply(tu_PROBABILITY)
tu_data['score_tu_SCORE_MB'] = tu_data.SCORE_MB.apply(tu_SCORE_MB)
tu_data['score_generic_score'] = tu_data.SCORE.apply(tu_SCORE_MB)
tu_data['score_tu_PASTDUEAMOUNT'] = tu_data.PASTDUEAMOUNT.apply(tu_PASTDUEAMOUNT)
tu_data['TU_score'] = tu_data['score_tu_NONPERFORMING'] + tu_data['score_tu_ARREARS0DAYS'] + tu_data['score_tu_PROBABILITY'] + tu_data['score_tu_SCORE_MB'] + tu_data['score_tu_PASTDUEAMOUNT']

In [288]:
tu_data

,CRN,RETRODATE,FIRSTNAME,SURNAME,NATIONALID,PASSPORTNO,MOBILENO,SERVICEID,ALIENID,DATEOFBIRTH,REGISTRATIONNUMBER,REGISTRATIONDATE,EMPLOYER,EMPLOYMENTPERIOD,INCOME,INCOMEFREQUENCY,HOUSETYPE,PHYSICALADDRESS,NUMBEROFPHYSICALADDRESSES,PHYSICALADDRESSLASTUPDATEDATE,DURATIONATPHYSICALADDRESS,NUMBEROFPOSTALADDRESSES,POSTALADDRESSLASTUPDATEDATE,NUMBEROFPHONECONTACTS,PHONECONTACTSLASTUPDATEDATE,FIRSTLISTEDDATE,NUMBEROF_ACCOUNTS_,INSTALMENTACCOUNT,REVOLVINGACCOUNT,OPENACCOUNT,CREDITCARD,HOMELOAN,SINGLECREDITFACILITY,OVERDRAFT,PENSIONBACKED,STUDENTLOAN,COLLECTIONACCOUNT,LINEOFCREDIT,PERFORMING,NONPERFORMING,CLOSED,CURRENTINARREARS,MAXARREARS,ARREARS0DAYS,ARREARS30DAYS,ARREARS60DAYS,ARREARS90DAYS,ARREARSGT90DAYS,EARLIESTDATEOPENED,PRINCIPALAMOUNT,SUBSCRIBERPRINCIPALAMOUNT,SCHEDULEDPAYMENTAMT,COLLATERALSGIVEN,CREDITAPPLICATIONSDONE,FRAUDCOUNT,CURRENTBALANCEAMOUNT,SUBSCRIBERCURRENTBALANCEAMOUNT,PASTDUEAMOUNT,LASTPAYMENTDATE,X_ACCOUNTS_DISPUTED,NOOFCHEQUESRETURNED,EARLIESTCHEQUERETURNDATE,BOUNCEDCHEQUEAMOUNT,NODDACS,EARLIESTDDACBOUNCEDATE,DDACBOUNCEDAMOUNT,NUMBEROFENQUIRIES,LASTENQUIRYDATE,ENQUIRIES30DAYS,ENQUIRIES60DAYS,ENQUIRIES90DAYS,ENQUIRIESGT90DAYS,SCORE,SCOREGRADE,PROBABILITY,SCORE_MB,SCOREGRADE_MB,PROBABILITY_MB,CONSUMERTYPE,CLIENTID,AARC1,AARC2,AARC3,AARC4,AARCM1,AARCM2,AARCM3,AARCM4,MOBILE_TOTAL,MOBILE_PERFORMING,MOBILE_NONPERFORMING,MOBILE_CLOSED,MOBILE_CURRENTINARREARS,MOBILE_PRINCIPALAMOUNT,MOBILE_SUBSCRIBERPRINCIPAL,MOBILE_CURRENTBALANCEAMOUNT,MOBILE_SUBSCRIBERCURRENTBALANCEAMOUNT,MOBILE_LASTPAYMENTDATE,score_tu_NONPERFORMING,score_tu_ARREARS0DAYS,score_tu_PROBABILITY,score_tu_SCORE_MB,score_generic_score,score_tu_PASTDUEAMOUNT,TU_score
0,7574778,30-10-2019,BERNARD WAMICHI,KIMUHU,5176836,NaN,NaN,NaN,NaN,1951-11-11 00:00:00,NaN,NaN,NIC BANK,296.0,0.0,NaN,NaN,P.O BOX 51324,4.0,13-Mar-2019,0.0,8.0,13-Mar-2019,3.0,NaN,12-05-2014,12.0,0.0,0.0,5.0,3.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,12.0,0.0,7.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,NaN,9.362806e+06,5500343.0,1.375656e+05,5.0,1.0,0.0,3916462.71,3596088.63,0.00,11-10-2019,NaN,0.0,NaN,0.0,0.0,NaN,0.0,8.0,26-07-2017,0.0,0.0,0.0,8.0,712,AA,5.03,663.0,BB,13.54,INDIVIDUAL,NaN,Many loan applications in the recent past (for...,"High number of open, longterm loans where the...",Many unpaid installments or no loans that have...,Loan installments haven't been paid in the rec...,Low number of payments recently,Satisfactory accounts are relatively old,Recent delinquency on accounts,Average credit limit of Revolving accounts are...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,NaN,140,-20,102,56,0,0,278
1,3338743,30-10-2019,HANNA WAMBUI,MBIRIRI,5197891,NaN,NaN,NaN,NaN,1931-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,09-Nov-2010,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.00,0.00,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,YY,999.99,0.0,YY,999.99,INDIVIDUAL,NaN,-,-,-,-,-,-,-,-,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,NaN,140,0,0,-39,0,0,101
2,7622950,30-10-2019,MUCHAI IHUTHIA,MURIU,5334407,NaN,NaN,NaN,NaN,1949-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,14-Sep-2019,0.0,3.0,15-Mar-2019,2.0,NaN,18-03-2017,36.0,0.0,0.0,19.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,36.0,0.0,17.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,NaN,3.727327e+05,0.0,7.129940e+04,10.0,10.0,0.0,43877.65,0.00,0.00,24-10-2019,NaN,0.0,NaN,0.0,0.0,NaN,0.0,3.0,21-06-2017,0.0,0.0,0.0,3.0,517,II,44.25,695.0,AA,9.37,INDIVIDUAL,NaN,Average credit limit for loans is relatively low,Borrowed a lot on loans other than secured loa...,Many loan applications in the recent past (for...,Many unpaid installments or no loans that have...,Average credit limit of accounts are relativel...,Low number of payments recently,Satisfactory accounts are relatively old,Recent delinquency on accounts,14.0,14.0,0.0,3.0,0.0,63828.35,0.0,22028.350,0.0,2019-05-20,140,14,3,0,0,0,157
3,328298,30-10-2019,BEATRICE WANGARI,MWANGI,5379876,NaN,NaN,NaN,NaN,1962-01-01 00

In [308]:
tu_data_1 = tu_data[tu_data.TU_score > 50 ]
tu_data_1

,CRN,RETRODATE,FIRSTNAME,SURNAME,NATIONALID,PASSPORTNO,MOBILENO,SERVICEID,ALIENID,DATEOFBIRTH,REGISTRATIONNUMBER,REGISTRATIONDATE,EMPLOYER,EMPLOYMENTPERIOD,INCOME,INCOMEFREQUENCY,HOUSETYPE,PHYSICALADDRESS,NUMBEROFPHYSICALADDRESSES,PHYSICALADDRESSLASTUPDATEDATE,DURATIONATPHYSICALADDRESS,NUMBEROFPOSTALADDRESSES,POSTALADDRESSLASTUPDATEDATE,NUMBEROFPHONECONTACTS,PHONECONTACTSLASTUPDATEDATE,FIRSTLISTEDDATE,NUMBEROF_ACCOUNTS_,INSTALMENTACCOUNT,REVOLVINGACCOUNT,OPENACCOUNT,CREDITCARD,HOMELOAN,SINGLECREDITFACILITY,OVERDRAFT,PENSIONBACKED,STUDENTLOAN,COLLECTIONACCOUNT,LINEOFCREDIT,PERFORMING,NONPERFORMING,CLOSED,CURRENTINARREARS,MAXARREARS,ARREARS0DAYS,ARREARS30DAYS,ARREARS60DAYS,ARREARS90DAYS,ARREARSGT90DAYS,EARLIESTDATEOPENED,PRINCIPALAMOUNT,SUBSCRIBERPRINCIPALAMOUNT,SCHEDULEDPAYMENTAMT,COLLATERALSGIVEN,CREDITAPPLICATIONSDONE,FRAUDCOUNT,CURRENTBALANCEAMOUNT,SUBSCRIBERCURRENTBALANCEAMOUNT,PASTDUEAMOUNT,LASTPAYMENTDATE,X_ACCOUNTS_DISPUTED,NOOFCHEQUESRETURNED,EARLIESTCHEQUERETURNDATE,BOUNCEDCHEQUEAMOUNT,NODDACS,EARLIESTDDACBOUNCEDATE,DDACBOUNCEDAMOUNT,NUMBEROFENQUIRIES,LASTENQUIRYDATE,ENQUIRIES30DAYS,ENQUIRIES60DAYS,ENQUIRIES90DAYS,ENQUIRIESGT90DAYS,SCORE,SCOREGRADE,PROBABILITY,SCORE_MB,SCOREGRADE_MB,PROBABILITY_MB,CONSUMERTYPE,CLIENTID,AARC1,AARC2,AARC3,AARC4,AARCM1,AARCM2,AARCM3,AARCM4,MOBILE_TOTAL,MOBILE_PERFORMING,MOBILE_NONPERFORMING,MOBILE_CLOSED,MOBILE_CURRENTINARREARS,MOBILE_PRINCIPALAMOUNT,MOBILE_SUBSCRIBERPRINCIPAL,MOBILE_CURRENTBALANCEAMOUNT,MOBILE_SUBSCRIBERCURRENTBALANCEAMOUNT,MOBILE_LASTPAYMENTDATE,score_tu_NONPERFORMING,score_tu_ARREARS0DAYS,score_tu_PROBABILITY,score_tu_SCORE_MB,score_generic_score,score_tu_PASTDUEAMOUNT,TU_score
0,7574778,30-10-2019,BERNARD WAMICHI,KIMUHU,5176836,NaN,NaN,NaN,NaN,1951-11-11 00:00:00,NaN,NaN,NIC BANK,296.0,0.0,NaN,NaN,P.O BOX 51324,4.0,13-Mar-2019,0.0,8.0,13-Mar-2019,3.0,NaN,12-05-2014,12.0,0.0,0.0,5.0,3.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,12.0,0.0,7.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,NaN,9.362806e+06,5500343.00,1.375656e+05,5.0,1.0,0.0,3916462.71,3596088.63,0.00,11-10-2019,NaN,0.0,NaN,0.0,0.0,NaN,0.0,8.0,26-07-2017,0.0,0.0,0.0,8.0,712,AA,5.03,663.0,BB,13.54,INDIVIDUAL,NaN,Many loan applications in the recent past (for...,"High number of open, longterm loans where the...",Many unpaid installments or no loans that have...,Loan installments haven't been paid in the rec...,Low number of payments recently,Satisfactory accounts are relatively old,Recent delinquency on accounts,Average credit limit of Revolving accounts are...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,NaN,140,-20,102,56,0,0,278
1,3338743,30-10-2019,HANNA WAMBUI,MBIRIRI,5197891,NaN,NaN,NaN,NaN,1931-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,09-Nov-2010,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000e+00,0.00,0.000000e+00,0.0,0.0,0.0,0.00,0.00,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,YY,999.99,0.0,YY,999.99,INDIVIDUAL,NaN,-,-,-,-,-,-,-,-,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,NaN,140,0,0,-39,0,0,101
2,7622950,30-10-2019,MUCHAI IHUTHIA,MURIU,5334407,NaN,NaN,NaN,NaN,1949-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,14-Sep-2019,0.0,3.0,15-Mar-2019,2.0,NaN,18-03-2017,36.0,0.0,0.0,19.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,36.0,0.0,17.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,NaN,3.727327e+05,0.00,7.129940e+04,10.0,10.0,0.0,43877.65,0.00,0.00,24-10-2019,NaN,0.0,NaN,0.0,0.0,NaN,0.0,3.0,21-06-2017,0.0,0.0,0.0,3.0,517,II,44.25,695.0,AA,9.37,INDIVIDUAL,NaN,Average credit limit for loans is relatively low,Borrowed a lot on loans other than secured loa...,Many loan applications in the recent past (for...,Many unpaid installments or no loans that have...,Average credit limit of accounts are relativel...,Low number of payments recently,Satisfactory accounts are relatively old,Recent delinquency on accounts,14.0,14.0,0.0,3.0,0.0,63828.35,0.0,22028.35,0.0,2019-05-20,140,14,3,0,0,0,157
3,328298,30-10-2019,BEATRICE WANGARI,MWANGI,5379876,NaN,NaN,NaN,NaN,1962-01-01 00

In [309]:
#elimination of age
from datetime import date 
  
def calculateAge(birthDate): 
    today = date.today() 
    age = today.year - birthDate.year - ((today.month, today.day) <  (birthDate.month, birthDate.day)) 
  
    return age 

In [310]:
tu_data_1['DATEOFBIRTH'] = pd.to_datetime( tu_data_1['DATEOFBIRTH'])

/Users/allan.aswani/etl/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [311]:
tu_data_1["age_cust"] = tu_data_1.DATEOFBIRTH.apply(calculateAge)

/Users/allan.aswani/etl/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [312]:
tu_data_1.dtypes


CRN                                               int64
RETRODATE                                        object
FIRSTNAME                                        object
SURNAME                                          object
NATIONALID                                       object
PASSPORTNO                                       object
MOBILENO                                        float64
SERVICEID                                        object
ALIENID                                         float64
DATEOFBIRTH                              datetime64[ns]
REGISTRATIONNUMBER                               object
REGISTRATIONDATE                                 object
EMPLOYER                                         object
EMPLOYMENTPERIOD                                float64
INCOME                                          float64
INCOMEFREQUENCY                                 float64
HOUSETYPE                                        object
PHYSICALADDRESS                                 

In [313]:
final_tu_scored_data = tu_data_1[tu_data_1.age_cust > 25]

In [314]:
final_tu_scored_data

,CRN,RETRODATE,FIRSTNAME,SURNAME,NATIONALID,PASSPORTNO,MOBILENO,SERVICEID,ALIENID,DATEOFBIRTH,REGISTRATIONNUMBER,REGISTRATIONDATE,EMPLOYER,EMPLOYMENTPERIOD,INCOME,INCOMEFREQUENCY,HOUSETYPE,PHYSICALADDRESS,NUMBEROFPHYSICALADDRESSES,PHYSICALADDRESSLASTUPDATEDATE,DURATIONATPHYSICALADDRESS,NUMBEROFPOSTALADDRESSES,POSTALADDRESSLASTUPDATEDATE,NUMBEROFPHONECONTACTS,PHONECONTACTSLASTUPDATEDATE,FIRSTLISTEDDATE,NUMBEROF_ACCOUNTS_,INSTALMENTACCOUNT,REVOLVINGACCOUNT,OPENACCOUNT,CREDITCARD,HOMELOAN,SINGLECREDITFACILITY,OVERDRAFT,PENSIONBACKED,STUDENTLOAN,COLLECTIONACCOUNT,LINEOFCREDIT,PERFORMING,NONPERFORMING,CLOSED,CURRENTINARREARS,MAXARREARS,ARREARS0DAYS,ARREARS30DAYS,ARREARS60DAYS,ARREARS90DAYS,ARREARSGT90DAYS,EARLIESTDATEOPENED,PRINCIPALAMOUNT,SUBSCRIBERPRINCIPALAMOUNT,SCHEDULEDPAYMENTAMT,COLLATERALSGIVEN,CREDITAPPLICATIONSDONE,FRAUDCOUNT,CURRENTBALANCEAMOUNT,SUBSCRIBERCURRENTBALANCEAMOUNT,PASTDUEAMOUNT,LASTPAYMENTDATE,X_ACCOUNTS_DISPUTED,NOOFCHEQUESRETURNED,EARLIESTCHEQUERETURNDATE,BOUNCEDCHEQUEAMOUNT,NODDACS,EARLIESTDDACBOUNCEDATE,DDACBOUNCEDAMOUNT,NUMBEROFENQUIRIES,LASTENQUIRYDATE,ENQUIRIES30DAYS,ENQUIRIES60DAYS,ENQUIRIES90DAYS,ENQUIRIESGT90DAYS,SCORE,SCOREGRADE,PROBABILITY,SCORE_MB,SCOREGRADE_MB,PROBABILITY_MB,CONSUMERTYPE,CLIENTID,AARC1,AARC2,AARC3,AARC4,AARCM1,AARCM2,AARCM3,AARCM4,MOBILE_TOTAL,MOBILE_PERFORMING,MOBILE_NONPERFORMING,MOBILE_CLOSED,MOBILE_CURRENTINARREARS,MOBILE_PRINCIPALAMOUNT,MOBILE_SUBSCRIBERPRINCIPAL,MOBILE_CURRENTBALANCEAMOUNT,MOBILE_SUBSCRIBERCURRENTBALANCEAMOUNT,MOBILE_LASTPAYMENTDATE,score_tu_NONPERFORMING,score_tu_ARREARS0DAYS,score_tu_PROBABILITY,score_tu_SCORE_MB,score_generic_score,score_tu_PASTDUEAMOUNT,TU_score,age_cust
0,7574778,30-10-2019,BERNARD WAMICHI,KIMUHU,5176836,NaN,NaN,NaN,NaN,1951-11-11,NaN,NaN,NIC BANK,296.0,0.0,NaN,NaN,P.O BOX 51324,4.0,13-Mar-2019,0.0,8.0,13-Mar-2019,3.0,NaN,12-05-2014,12.0,0.0,0.0,5.0,3.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,12.0,0.0,7.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,NaN,9.362806e+06,5500343.00,1.375656e+05,5.0,1.0,0.0,3916462.71,3596088.63,0.00,11-10-2019,NaN,0.0,NaN,0.0,0.0,NaN,0.0,8.0,26-07-2017,0.0,0.0,0.0,8.0,712,AA,5.03,663.0,BB,13.54,INDIVIDUAL,NaN,Many loan applications in the recent past (for...,"High number of open, longterm loans where the...",Many unpaid installments or no loans that have...,Loan installments haven't been paid in the rec...,Low number of payments recently,Satisfactory accounts are relatively old,Recent delinquency on accounts,Average credit limit of Revolving accounts are...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,NaN,140,-20,102,56,0,0,278,67.0
1,3338743,30-10-2019,HANNA WAMBUI,MBIRIRI,5197891,NaN,NaN,NaN,NaN,1931-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,09-Nov-2010,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000e+00,0.00,0.000000e+00,0.0,0.0,0.0,0.00,0.00,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,YY,999.99,0.0,YY,999.99,INDIVIDUAL,NaN,-,-,-,-,-,-,-,-,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,NaN,140,0,0,-39,0,0,101,88.0
2,7622950,30-10-2019,MUCHAI IHUTHIA,MURIU,5334407,NaN,NaN,NaN,NaN,1949-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,14-Sep-2019,0.0,3.0,15-Mar-2019,2.0,NaN,18-03-2017,36.0,0.0,0.0,19.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,36.0,0.0,17.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,NaN,3.727327e+05,0.00,7.129940e+04,10.0,10.0,0.0,43877.65,0.00,0.00,24-10-2019,NaN,0.0,NaN,0.0,0.0,NaN,0.0,3.0,21-06-2017,0.0,0.0,0.0,3.0,517,II,44.25,695.0,AA,9.37,INDIVIDUAL,NaN,Average credit limit for loans is relatively low,Borrowed a lot on loans other than secured loa...,Many loan applications in the recent past (for...,Many unpaid installments or no loans that have...,Average credit limit of accounts are relativel...,Low number of payments recently,Satisfactory accounts are relatively old,Recent delinquency on accounts,14.0,14.0,0.0,3.0,0.0,63828.35,0.0,22028.35,0.0,2019-05-20,140,14,3,0,0,0,157,70.0
3,328298,30-10-2019,BEATRICE WANGARI,MWANGI,5379876,NaN,NaN,NaN,NaN,1962-01-01,NaN,N

In [319]:
#export_csv = lendable_cust_2.to_excel (r'/Users/allan.aswani/Documents/scorecardworks/all_cust_data_repeat_score_card_data_tu_version', header=True) #Don't forget to add '.csv' at the end of the path

final_tu_scored_data.to_csv(r'/Users/allan.aswani/Documents/scorecardworks/tu_version_after_accounting_age_crb_data.csv', header=True)


In [318]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [326]:
final_tu_scored_data.to_excel(r'/Users/allan.aswani/Documents/scorecardworks/tu_version_after_accounting_age_crb_data.xlsx')